In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, log_loss, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
from scipy.stats import kurtosis
import time
import warnings
import lightgbm as lgb
warnings.filterwarnings('ignore')

from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)

In [ ]:
train_df = pd.read_csv('dataset/train.csv', parse_dates=['auditing_date', 'due_date', 'repay_date'])
train_df['repay_date'] = train_df[['due_date', 'repay_date']].apply(lambda x: x['repay_date'] if x['repay_date'] != '\\N' else x['due_date'], axis=1)
train_df['repay_amt'] = train_df['repay_amt'].apply(lambda x: x if x != '\\N' else 0).astype('float32')
train_df['label'] = (train_df['due_date'] - train_df['repay_date']).dt.days
train_df.loc[train_df['repay_amt'] == 0, 'label'] = 32
clf_labels = train_df['label'].values
amt_labels = train_df['repay_amt'].values
del train_df['label'], train_df['repay_amt'], train_df['repay_date']
train_due_amt_df = train_df[['due_amt']]
train_num = train_df.shape[0]
test_df = pd.read_csv('dataset/test.csv', parse_dates=['auditing_date', 'due_date'])
sub = test_df[['listing_id', 'auditing_date', 'due_amt']]
df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

listing_info_df = pd.read_csv('dataset/listing_info.csv')
listing_info_df['auditing_date_month'] = pd.to_datetime(listing_info_df['auditing_date']).dt.month
listing_info_df['auditing_date_days'] = listing_info_df['auditing_date_month'].copy()
# print(sub_example['due_date'])
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==1] = 31
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==3] = 31
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==5] = 31
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==7] = 31
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==8] = 31
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==10] = 31
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==12] = 31
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==4] = 30
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==6] = 30
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==9] = 30
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==11] = 30
listing_info_df['auditing_date_days'][listing_info_df['auditing_date_days']==2] = 28

listing_info_df['principal_per_term'] = listing_info_df['principal']/listing_info_df['term']
list_count=listing_info_df['user_id'].value_counts()
df_count_times=pd.DataFrame(list_count)
df_count_times.rename(columns={ df_count_times.columns[0]: "loan_times" },inplace=True)
df_count_times['user_id']=df_count_times.index
bin_size=(1,2,4,8)
def count_to_style(count):
    n=1
    num=1
    while count>n and num<5:
        n=n*2
        num=num+1
    return num
df_count_times['loan_times_type']=df_count_times['loan_times'].apply(count_to_style)
del df_count_times['loan_times']


list_principle=listing_info_df['principal'].value_counts()
principal_df=pd.DataFrame(list_principle)
principal_df.rename(columns={principal_df.columns[0]: "frequency"},inplace=True)
principal_df['principal']=principal_df.index
principal_df.reset_index(drop=True, inplace=True)
def ChangeToClass(principal):
    if principal<=1170:
        return 6
    elif 1170<principal<=2260:
        return 5
    elif 2260<principal<=3350:
        return 4
    elif 3350<principal<=3910:
        return 3
    elif 3910<principal<=5540:
        return 2
    else:
        return 1
principal_df['principal_class']=principal_df['principal'].apply(ChangeToClass)
del principal_df['frequency']

listing_info_df=listing_info_df.merge(principal_df, on="principal", how="left")

listing_info_df['TermRate']=listing_info_df['rate']/listing_info_df['term']

print(listing_info_df.head())
del listing_info_df['user_id'], listing_info_df['auditing_date']
df = df.merge(listing_info_df, on='listing_id', how='left')

# 表中有少数user不止一条记录，因此按日期排序，去重，只保留最新的一条记录。
user_info = pd.read_csv('dataset/user_info.csv', parse_dates=['reg_mon', 'insertdate'])

user_info.rename(columns={'insertdate': 'info_insert_date'}, inplace=True)

user_info = user_info.sort_values(by='info_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)

user_info['gender']=user_info['gender'].apply(lambda x:1 if x.strip()=='男' else 0)

user_info['id_city'] = user_info['id_city'].apply(lambda x: x.replace('\\N', 'c0'))
def compare(a,b):
    if a==b: return 1
    else: return 0
user_info['remote_boolean']=user_info.apply(lambda user_info:compare(user_info['cell_province'],user_info['id_province']),axis=1)

city_map=user_info.groupby('id_city').agg({'id_city':'count'})
city_map[city_map['id_city']>11000]
city_map.columns=['count']
city_map.reset_index(inplace=True)
x = city_map[city_map["count"] < 11000]; 
max_count = city_map["count"].max()
min_count = city_map["count"].min()

bin_size = int((max_count - min_count) / 5); 
def count_to_class(count): 
    return min(6, int((count - min_count) / bin_size))+1
city_map['class']=city_map['count'].apply(count_to_class)
city_map["class"][0]=0
city_map = city_map[["id_city", "class"]]
user_info=user_info.merge(city_map, on="id_city", how="left")

user_info=user_info.merge(df_count_times, on="user_id", how="left")  # 合并贷款次数到user_info里面

df = df.merge(user_info, on='user_id', how='left')

# 同上
user_tag_df = pd.read_csv('dataset/user_taglist.csv', parse_dates=['insertdate'])
user_tag_df.rename(columns={'insertdate': 'tag_insert_date'}, inplace=True)
user_tag_df = user_tag_df.sort_values(by='tag_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)
df = df.merge(user_tag_df, on='user_id', how='left')

# 历史记录表能做的特征远不止这些
repay_log_df = pd.read_csv('dataset/user_repay_logs.csv', parse_dates=['due_date', 'repay_date'])

# 由于题目任务只预测第一期的还款情况，因此这里只保留第一期的历史记录。当然非第一期的记录也能提取很多特征。
repay_log_df = repay_log_df[repay_log_df['order_id'] == 1].reset_index(drop=True)
repay_log_df['repay'] = repay_log_df['repay_date'].astype('str').apply(lambda x: 1 if x != '2200-01-01' else 0)
repay_log_df['early_repay_days'] = (repay_log_df['due_date'] - repay_log_df['repay_date']).dt.days
repay_log_df['early_repay_days'] = repay_log_df['early_repay_days'].apply(lambda x: x if x >= 0 else -1)
for f in ['listing_id', 'order_id', 'due_date', 'repay_date', 'repay_amt']:
    del repay_log_df[f]

group = repay_log_df.groupby('user_id', as_index=False)
repay_log_df = repay_log_df.merge(group['repay'].agg({'repay_mean': 'mean'}), on='user_id', how='left')

repay_log_df = repay_log_df.merge(
    group['early_repay_days'].agg({
        'early_repay_days_max': 'max', 'early_repay_days_median': 'median', 'early_repay_days_sum': 'sum',
        'early_repay_days_mean': 'mean', 'early_repay_days_std': 'std'
    }), on='user_id', how='left'
)

repay_log_df = repay_log_df.merge(
    group['due_amt'].agg({
        'due_amt_max': 'max', 'due_amt_min': 'min', 'due_amt_median': 'median',
        'due_amt_mean': 'mean', 'due_amt_sum': 'sum', 'due_amt_std': 'std',
        'due_amt_skew': 'skew', 'due_amt_kurt': kurtosis, 'due_amt_ptp': np.ptp
    }), on='user_id', how='left'
)
del repay_log_df['repay'], repay_log_df['early_repay_days'], repay_log_df['due_amt']
repay_log_df = repay_log_df.drop_duplicates('user_id').reset_index(drop=True)
df = df.merge(repay_log_df, on='user_id', how='left')

repay_log_df_all = pd.read_csv('dataset/user_repay_logs.csv', parse_dates=['due_date', 'repay_date'])
repay_log_df_all['repay_all_order'] = repay_log_df_all['repay_date'].astype('str').apply(lambda x: 1 if x != '2200-01-01' else 0)
repay_log_df_all['early_repay_days_all_order'] = (repay_log_df_all['due_date'] - repay_log_df_all['repay_date']).dt.days
repay_log_df_all['early_repay_days_all_order'] = repay_log_df_all['early_repay_days_all_order'].apply(lambda x: x if x >= 0 else -1)
for f in ['listing_id', 'order_id', 'due_date', 'repay_date', 'repay_amt']:
    del repay_log_df_all[f]

group = repay_log_df_all.groupby('user_id', as_index=False)
repay_log_df_all = repay_log_df_all.merge(group['repay_all_order'].agg({'repay_mean_all_order': 'mean'}), on='user_id', how='left')

repay_log_df_all = repay_log_df_all.merge(
    group['early_repay_days_all_order'].agg({
        'early_repay_days_max_all_order': 'max', 'early_repay_days_median_all_order': 'median', 'early_repay_days_sum_all_order': 'sum',
        'early_repay_days_mean_all_order': 'mean', 'early_repay_days_std_all_order': 'std'
    }), on='user_id', how='left'
)

repay_log_df_all = repay_log_df_all.merge(
    group['due_amt'].agg({
        'due_amt_max_all_order': 'max', 'due_amt_min_all_order': 'min', 'due_amt_median_all_order': 'median',
        'due_amt_mean_all_order': 'mean', 'due_amt_sum_all_order': 'sum', 'due_amt_std_all_order': 'std',
        'due_amt_skew_all_order': 'skew', 'due_amt_kurt_all_order': kurtosis, 'due_amt_ptp_all_order': np.ptp
    }), on='user_id', how='left'
)
del repay_log_df_all['repay_all_order'], repay_log_df_all['early_repay_days_all_order'], repay_log_df_all['due_amt']
repay_log_df_all = repay_log_df_all.drop_duplicates('user_id').reset_index(drop=True)
df = df.merge(repay_log_df_all, on='user_id', how='left')


cate_cols = ['cell_province', 'id_province', 'id_city']
for f in cate_cols:
    df[f] = df[f].map(dict(zip(df[f].unique(), range(df[f].nunique())))).astype('int32')

df['due_amt_per_days'] = df['due_amt'] / (train_df['due_date'] - train_df['auditing_date']).dt.days
date_cols = ['auditing_date', 'due_date', 'reg_mon', 'info_insert_date', 'tag_insert_date']

for f in date_cols:
    if f in ['reg_mon', 'info_insert_date', 'tag_insert_date']:
        df[f + '_year'] = df[f].dt.year
    df[f + '_month'] = df[f].dt.month
    if f in ['auditing_date', 'due_date', 'info_insert_date', 'tag_insert_date']:
        df[f + '_day'] = df[f].dt.day
        df[f + '_dayofweek'] = df[f].dt.dayofweek

df.drop(columns=date_cols, axis=1, inplace=True)

# df['taglist'] = df['taglist'].astype('str').apply(lambda x: x.strip().replace('|', ' ').strip())
# tag_cv = CountVectorizer(min_df=10, max_df=0.9).fit_transform(df['taglist'])

del df['user_id'], df['listing_id'], df['taglist']
df_count_times
df = pd.get_dummies(df, columns=cate_cols)
# df = sparse.hstack((df.values, tag_cv), format='csr', dtype='float32')
# train_values, test_values = df[:train_num].values, df[train_num:].values
# print(train_values.shape)


   user_id  listing_id auditing_date  term  rate  principal  \
0   316610     1556649    2017-11-26     9   7.6       4800   
1    62002     1556633    2017-11-26     6   7.6       4000   
2   192135     1556629    2017-11-26    12   8.0       8660   
3   487382     1556628    2017-11-26     9   7.6       4780   
4   235186     1556627    2017-11-26     9   7.6       1480   

   auditing_date_month  auditing_date_days  principal_per_term  \
0                   11                  30          533.333333   
1                   11                  30          666.666667   
2                   11                  30          721.666667   
3                   11                  30          531.111111   
4                   11                  30          164.444444   

   principal_class  TermRate  
0                2  0.844444  
1                2  1.266667  
2                1  0.666667  
3                2  0.844444  
4                5  0.844444  


In [ ]:
train_values, test_values = df[:train_num].values, df[train_num:].values
# 五折验证也可以改成一次验证，按时间划分训练集和验证集，以避免由于时序引起的数据穿越问题。
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'metric_freq': 5,
    'num_class': 33,
    'num_iterations': 600,
    'num_leaves': 31,
    'learning_rate': 0.05,
#     'device_type':'gpu'
}
amt_oof = np.zeros(train_num)
prob_oof = np.zeros((train_num, 33))
test_pred_prob = np.zeros((test_values.shape[0], 33))


for i, (trn_idx, val_idx) in enumerate(skf.split(train_values, clf_labels)):
    print(i, 'fold...')
    t = time.time()
    trn_x, trn_y = train_values[trn_idx], clf_labels[trn_idx]
    val_x, val_y = train_values[val_idx], clf_labels[val_idx]
    val_repay_amt = amt_labels[val_idx]
    val_due_amt = train_due_amt_df.iloc[val_idx]
   
    train = lgb.Dataset(trn_x, label=trn_y)
    val = lgb.Dataset(val_x, label=val_y, reference=train)
    gbm = lgb.train(params, train, valid_sets=val, early_stopping_rounds=10)

    # shepe = (-1, 33)
    val_pred_prob_everyday = gbm.predict(val_x, num_iteration=gbm.best_iteration)
    prob_oof[val_idx] = val_pred_prob_everyday
    val_pred_prob_today = [val_pred_prob_everyday[i][val_y[i]] for i in range(val_pred_prob_everyday.shape[0])]
    val_pred_repay_amt = val_due_amt['due_amt'].values * val_pred_prob_today
    print('val rmse:', np.sqrt(mean_squared_error(val_repay_amt, val_pred_repay_amt)))
    print('val mae:', mean_absolute_error(val_repay_amt, val_pred_repay_amt))
    amt_oof[val_idx] = val_pred_repay_amt
    test_pred_prob += gbm.predict(test_values, num_iteration=gbm.best_iteration) / skf.n_splits
    print('runtime: {}\n'.format(time.time() - t))

print('\ncv rmse:', np.sqrt(mean_squared_error(amt_labels, amt_oof)))
print('cv mae:', mean_absolute_error(amt_labels, amt_oof))
print('cv logloss:', log_loss(clf_labels, prob_oof))
print('cv acc:', accuracy_score(clf_labels, np.argmax(prob_oof, axis=1)))

prob_cols = ['prob_{}'.format(i) for i in range(33)]

for i, f in enumerate(prob_cols):
    sub[f] = test_pred_prob[:, i]

sub_example = pd.read_csv('dataset/submission.csv', parse_dates=['repay_date'])

sub_example = sub_example.merge(sub, on='listing_id', how='left')
# sub_example['due_date'] = pd.to_datetime((sub_example['auditing_date'] + np.timedelta64(1, 'M') + np.timedelta64(1, 'D')).dt.date)
sub_example['due_date'] = sub_example['auditing_date'].copy()
# print(sub_example['due_date'])
sub_example['due_date'][sub_example['due_date'].dt.month == 4] = pd.to_datetime((sub_example['due_date'][sub_example['due_date'].dt.month == 4] + np.timedelta64(1, 'M')).dt.date)
sub_example['due_date'][sub_example['due_date'].dt.month == 3] = pd.to_datetime((sub_example['due_date'][sub_example['due_date'].dt.month == 3] + np.timedelta64(1, 'M') + np.timedelta64(1, 'D')).dt.date)
sub_example['due_date'][sub_example['due_date'].dt.month == 2] = pd.to_datetime((sub_example['due_date'][sub_example['due_date'].dt.month == 2] + np.timedelta64(1, 'M') - np.timedelta64(2, 'D')).dt.date)

sub_example['days'] = (sub_example['due_date'] - sub_example['repay_date']).dt.days

# shape = (-1, 33)
test_prob = sub_example[prob_cols].values
test_labels = sub_example['days'].values
test_prob = [test_prob[i][test_labels[i]] for i in range(test_prob.shape[0])]
sub_example['repay_amt'] = sub_example['due_amt'] * test_prob
sub_example[['listing_id', 'repay_date', 'repay_amt']].to_csv('sub_cross_5.csv', index=False)

In [1]:
range(33)

range(0, 33)

In [2]:
list(range(33))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32]